In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math as math
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
import sklearn

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
df=pd.read_csv('/content/drive/MyDrive/nga-east_rotd50_5pct_flatfile_public_20141118.csv')

In [ ]:
# df = df.drop(df[df['ClstD (km)'] == -999 ].index)
# df = df.drop(df[df['Preferred VS30 (m/sec)'] == -999 ].index)
df = df.drop_duplicates()
# df = df.drop(df[df['Quality Flag Based on Ground Motions Residuals'] == -999 ].index)
# df = df.drop(df[df['Depth Used (km)'] == -999 ].index)
# df = df.drop(df[df['Hypocenter Depth (km)'] == -999 ].index)
df = df.drop(df[df['Joyner-Boore Dist. (km)'] == -999 ].index)
df = df.drop(df[df['Joyner-Boore Dist. (km)'] > 1500 ].index)
df = df.drop(df[df['PGA-H RotDnn (g)'] == -999 ].index)
# df = df.drop(df[df['Station Latitude (deg)'] == -999 ].index)
# df = df.drop(df[df['Station Longitude (deg)'] == -999 ].index)
# df = df.drop(df[df['Earthquake Magnitude'] < 0 ].index)
df = df.drop(df[df['HypD (km)'] <= 0 ].index)

In [ ]:

rows, cols = df.shape

print("Number of rows:", rows)
print("Number of columns:", cols)

In [ ]:
df.reset_index()

In [ ]:
from keras.layers import Input, Dense, Lambda
from keras.models import Model
from keras import backend as K
from keras.losses import mse


In [ ]:

# Define VAE architecture


# def leaky_relu(x, alpha=0.2):
#     return tf.maximum(alpha * x, x)

def create_vae(input_dim, latent_dim, intermediate_dim,intermediate_dim_1,intermediate_dim_2,intermediate_dim_3):

    input_layer = Input(shape=(input_dim,))

    x = Dense(intermediate_dim, activation='relu')(input_layer)
    x = Dense(intermediate_dim_1, activation='relu')(x)
    x = Dense(intermediate_dim_2, activation='relu')(x)
    x = Dense(intermediate_dim_3, activation='relu')(x)
    # x = Dense(intermediate_dim_4, activation='sigmoid')(x)
    # x = Dense(intermediate_dim_5, activation='sigmoid')(x)
    # x = Dense(intermediate_dim_6, activation='sigmoid')(x)



    z_mean = Dense(latent_dim)(x)
    z_log_var = Dense(latent_dim)(x)


    def sampling(args):
        z_mean, z_log_var = args
        epsilon = K.random_normal(shape=(K.shape(z_mean)[0], latent_dim), mean=0.0, stddev=1.0)
        return z_mean + K.exp(0.5 * z_log_var) * epsilon


    z = Lambda(sampling)([z_mean, z_log_var])


    # decoder_h = Dense(intermediate_dim_6, activation='sigmoid')
    # decoder_h = Dense(intermediate_dim_5, activation='sigmoid')
    # decoder_h = Dense(intermediate_dim_4, activation='sigmoid')
    decoder_h = Dense(intermediate_dim_3, activation='relu')
    decoder_h = Dense(intermediate_dim_2, activation='relu')
    decoder_h = Dense(intermediate_dim_1, activation='relu')
    decoder_h = Dense(intermediate_dim, activation='relu')
    decoder_mean = Dense(input_dim, name='decoder_output')  # Output layer with name 'decoder_output'

    h_decoded = decoder_h(z)
    x_decoded_mean = decoder_mean(h_decoded)


    vae = Model(input_layer, x_decoded_mean)


    reconstruction_loss = mse(K.flatten(input_layer), K.flatten(x_decoded_mean))
    kl_loss = - 0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)

    vae_loss = K.mean(reconstruction_loss + 0.1*kl_loss)
    # vae_loss = K.mean(reconstruction_loss)
    vae.add_loss(vae_loss)

    return vae, z_mean, z_log_var

# Example usage:
input_dim = y_train.shape[1]  # Dimension of input and output
latent_dim = 3   # Dimension of latent space
intermediate_dim = 20  # Dimension of intermediate layer
intermediate_dim_1 = 15
intermediate_dim_2 = 10
intermediate_dim_3 = 6
# intermediate_dim_6 = 4

vae, z_mean, z_log_var = create_vae(input_dim, latent_dim, intermediate_dim,intermediate_dim_1,intermediate_dim_2,intermediate_dim_3)
vae.compile(optimizer='adam', metrics = ['R2Score'])
# vae.compile(optimizer='adam', loss='mse', metrics = ['R2Score'])
vae.summary()


In [ ]:
vae.fit(y_train, y_train, epochs=250, batch_size=128, validation_data=(y_val,y_val))
vae.save('/content/drive/My Drive/SEISMIC MODELS/vae.h5')

In [ ]:
vae = tf.keras.models.load_model('/content/drive/My Drive/SEISMIC MODELS/vae.h5')

In [ ]:
# import numpy as np
y_pred_a = model_ann.predict(x_test)
y_pred_c = [f"log PSA {i}s" for i in t]
# y_pred_c = y_pred_a[:,[0,4,9,18]]
y_pred_c.append("log PGA");
y_pred_c.append("log PGV")
y_pred = pd.DataFrame(y_pred_a, columns=y_pred_c)

y_pred_a = vae.predict(y_pred)
y_pred_c = [f"log PSA {i}s" for i in t]
y_pred_c.append("log PGA")
y_pred_c.append("log PGV")
y_pred = pd.DataFrame(y_pred_a, columns=y_pred_c)

In [ ]:
from sklearn.metrics import r2_score

# Calculate R^2 score for the first dataframe
r2_score_1 = r2_score(y_test,y_pred)


print("R^2 score for dataframe 1:", r2_score_1)
